# Linearity in Finance

> To-do list:
* Examining the CAPM model with the efficient frontier and capital market line
* Solving for the security market line using regression
* Examining the APT model and performing a multivariate linear regression
* Understanding linear optimization in portfolio allocation
* Linear optimization using the PuLP package
* Understanding the outcomes of linear programming
* Introduction to integer programming
* Implementing a linear integer programming model with binary conditions
* Solving systems of linear equations with equalities using matrix linear algebra
* Solving systems of linear equations directly with the LU, Cholesky, and QR decomposition
* Solving systems of linear equations indirectly with the Jacobi and Gauss-Seidel method

## CAPM model

$$ R_i = R_f+\beta_i(R_{mkt} - R_f) $$

The `scipy.stats.linregress` function returns the slope of the regression line, the intercept of the regression line, the correlation coefficient, the p-value for a hypothesis test with null hypothesis of a zero slope, and the standard error of the estimate

| Time period | Stock returns | Market returns |
| ----------- | ----------- | ----------- |
| 1 | 0.065 | 0.055 |
|2 |0.0265|-0.09
|3|-0.0593|-0.041
|4|-0.001|0.045
|5|0.0346|0.022






In [3]:
""" Linear regression with SciPy """
from scipy import stats
stock_returns = [0.065, 0.0265, -0.0593, -0.001, 0.0346]
mkt_returns = [0.055, -0.09, -0.041, 0.045, 0.022]
beta, alpha, r_value, p_value, std_err = stats.linregress(stock_returns, mkt_returns)

print(beta, alpha)

0.5077431878770808 -0.008481900352462384


* The equation describing the SML can be written as:
$$ E(R_i)=R_f +β_i [E(R_M)−R_f] $$

## The Arbitrage Pricing Theory (APT) model

$$ E[R_i]=α_i+β_{i,1}F_1+β_{i,2}F_2+...+β_{i,j}F_j $$

In [16]:
""" Least squares regression with statsmodels """
import numpy as np
import statsmodels.api as sm
# Generate some sample data
num_periods = 9
# create a 9*8 array
# Each row represents a period, and each column represents a variable
all_values = np.array([np.random.random(8)
                        for i in range(num_periods)])
# Filter the data
y_values = all_values[:, 0]  # First column values as dependent variable (Y).
x_values = all_values[:, 1:]  # All other values as the independent variables (X).
x_values = sm.add_constant(x_values)  # Include the intercept
results = sm.OLS(y_values, x_values).fit()  # Regress and fit the model

In [18]:
results.summary()
results.params


/Users/shenyichen/opt/anaconda3/lib/python3.9/site-packages/scipy/stats/stats.py:1541: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=9
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


array([ 1.66165947,  0.08730198, -0.43483682, -0.49216373,  0.76991895,
       -0.63727415, -0.9484386 , -0.31855383])

## Linear optimization with PuLP

> A simple example:

Objective function:

 $$ Maximize { f(x,y)=3x+2y }$$

 Subject to: $$ 2 x + y ≤ 100 $$
 $$x + y ≤ 80 $$
 $$x ≤ 40$$
$$x ≥ 0, y ≥ 0 $$

In [23]:
""" A simple linear optimization problem with 2 variables """
import pulp
x = pulp.LpVariable("x", lowBound=0)
y = pulp.LpVariable("y", lowBound=0)
problem = pulp.LpProblem("A simple maximization objective",
                         pulp.LpMaximize)
problem += 3*x + 2*y, "The objective function"
problem += 2*x + y <= 100, "1st constraint"
problem += x + y <= 80, "2nd constraint"
problem += x <= 40, "3rd constraint"
problem.solve()

/Users/shenyichen/opt/anaconda3/lib/python3.9/site-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/shenyichen/opt/anaconda3/lib/python3.9/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/30/5npdvf4d7mb7d1df8ndd9dmm0000gn/T/415370241a0c4611a7af67119da3e906-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/30/5npdvf4d7mb7d1df8ndd9dmm0000gn/T/415370241a0c4611a7af67119da3e906-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 8 COLUMNS
At line 16 RHS
At line 20 BOUNDS
At line 21 ENDATA
Problem MODEL has 3 rows, 2 columns and 5 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 2 (-1) rows, 2 (0) columns and 4 (-1) elements
0  Obj -0 Dual inf 4.999998 (2)
2  Obj 180
Optimal - objective value 180
After Postsolve, objective 180, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 180 - 2 iterations time 0.002, Presolve 0.00
Option for printingOptions changed from norma

1

In [26]:
for variable in problem.variables():
    print (variable.name, "=", variable.varValue)

x = 20.0
y = 60.0


## Integer programming model with binary conditions

Suppose we must go for 150 contracts in a particular over-the-counter exotic security from three dealers. Dealer X quoted $500 per contract plus handling
fees of $4,000, regardless of the number of contracts sold. Dealer Y charges $450 per contract plus a transaction fee of $2,000. Dealer Z charges $450 per contract plus a fee of $6,000. Dealer X will sell at most 100 contracts, dealer Y at most 90, and dealer Z at most 70. The minimum transaction volume from any dealer is 30 contracts if any are transacted with that dealer. How should we minimize the cost of purchasing 150 contracts?



$$Minimize \sum_{i=x}^{i=z} IsOrder_i [variable\ cost_i × quantity_i + fixed\ cost_i ]$$


In [33]:
import pulp
dealers = ["X", "Y", "Z"]
variable_costs = {"X": 500,
                    "Y": 350,
                    "Z": 450}
fixed_costs = {"X": 4000,
                "Y": 2000,
                "Z": 6000}
# Define PuLP variables to solve
quantities = pulp.LpVariable.dicts("quantity",
                                    dealers,
                                    lowBound=0,
                                    cat=pulp.LpInteger)
is_orders = pulp.LpVariable.dicts("orders",
                                    dealers,
                                    cat=pulp.LpBinary)

In [35]:
# Initialize the model with constraints
model = pulp.LpProblem("A cost minimization problem", pulp.LpMinimize)
model += sum([variable_costs[i]*quantities[i] + fixed_costs[i]*is_orders[i] for i in dealers]), \
        "Minimize portfolio cost"
model += sum([quantities[i] for i in dealers]) == 150, \
        "Total contracts required"
model += is_orders["X"]*30 <= quantities["X"] <= is_orders["X"]*100, "Boundary of total volume of X"
model += is_orders["Y"]*30 <= quantities["Y"] <= is_orders["Y"]*90, "Boundary of total volume of Y"
model += is_orders["Z"]*30 <= quantities["Z"] <= is_orders["Z"]*70, "Boundary of total volume of Z"
model.solve()



Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/shenyichen/opt/anaconda3/lib/python3.9/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/30/5npdvf4d7mb7d1df8ndd9dmm0000gn/T/3e88d3e8f09f40fcba786944caec07b0-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/30/5npdvf4d7mb7d1df8ndd9dmm0000gn/T/3e88d3e8f09f40fcba786944caec07b0-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 9 COLUMNS
At line 37 RHS
At line 42 BOUNDS
At line 49 ENDATA
Problem MODEL has 4 rows, 6 columns and 9 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 65642.9 - 0.00 seconds
Cgl0004I processed model has 4 rows, 6 columns (6 integer (3 of which binary)) and 9 elements
Cutoff increment increased from 1e-05 to 49.9999
Cbc0038I Initial state - 1 integers unsatisfied sum - 0.142857
Cbc0038I Solution found of 66500
Cbc0038I Cleaned solutio

1

In [37]:
for variable in model.variables():
    print (variable.name, "=", variable.varValue)

print("total cost:", pulp.value(model.objective))

orders_X = 0.0
orders_Y = 1.0
orders_Z = 1.0
quantity_X = 0.0
quantity_Y = 90.0
quantity_Z = 60.0
total cost: 66500.0


## Solving linear equations using matrices


The linear equations can now be stated as follows:
$$ Ax = B $$

Solution: 
$$ x = A^{-1}B$$

In [39]:
import numpy as np
# create two arrays
A = np.array([[2, 1, 1],
                [1, 3, 2],
                [1, 0, 0]])
B = np.array([4, 5, 6])

np.linalg.solve(A, B )

array([  6.,  15., -23.])

## The LU decomposition

LU factorization decomposes matrix A into a product of two matrices: a lower triangular matrix L and an upper triangular matrix U. 

$$ A = L\ U $$



In [41]:
""" LU decomposition with SciPy """
import scipy.linalg as linalg
import numpy as np
A = np.array([[2., 1., 1.],
                [1., 3., 2.],
                [1., 0., 0.]])
B = np.array([4., 5., 6.])
LU = linalg.lu_factor(A)
x = linalg.lu_solve(LU, B)

print(x)


[  6.  15. -23.]
